In [ ]:
!pip install arviz

     |████████████████████████████████| 1.5MB 3.5MB/s 
     |████████████████████████████████| 727kB 22.3MB/s 
     |████████████████████████████████| 4.3MB 27.6MB/s 
     |████████████████████████████████| 296kB 32.7MB/s 
  Found existing installation: xarray 0.15.1
    Uninstalling xarray-0.15.1:
      Successfully uninstalled xarray-0.15.1


In [60]:
from scipy import stats
import numpy as np
import pandas as pd
import pymc3 as pm
#import pystan
#import arviz as az
import altair as alt

from IPython.display import Math, HTML

In [61]:
display(HTML("<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/"
               "latest.js?config=default'></script>"))

**2E1. Which of the expressions below correspond to the statement: the probability of rain on Monday?**

(2) Pr(rain|Monday) and (4) Pr(rain,Monday)/Pr(Monday) since

In [62]:
display(Math(r"Pr(rain,Monday) = Pr(rain\cap Monday)"))
display(Math(r"\frac{Pr(rain\cap Monday)}{Pr(Monday)} = \
               \frac{Pr(rain|Monday) * Pr(Monday)}{Pr(Monday)} = Pr(rain|Monday)")) 

<IPython.core.display.Math object>

<IPython.core.display.Math object>

**2E2. Which of the following statements corresponds to the expression: Pr(Monday|rain)?**


(3) The  probability that it is Monday, given that it is raining.

**2E3. Which of the expressions below correspond to the statement: the probability that it is Monday,
given that it is raining?**

(1) Pr(Monday|rain) and (4) Pr(rain|Monday)Pr(Monday)/Pr(rain) (Bayes rule in formula form)

**2E4. The Bayesian statistician Bruno de Finetti (1906–1985) began his book on probability theory with the declaration: “PROBABILITY DOES NOT EXIST.” The capitals appeared in the original, so I imagine de Finetti wanted us to shout this statement. What he meant is that probability is a device for describing uncertainty from the perspective of an observer with limited knowledge; it has no objective reality. Discuss the globe tossing example from the chapter, in light of this statement. What does it mean to say “the probability of water is 0.7”?**

Observer's knowledge in this example is limited by the experiment they ran and result they obtained. Probability in this sense means the belief about the "big world" build upon result produced by the "small world" manipulations. 

**2M1. Recall the globe tossing model from the chapter. Compute and plot the grid approximate posterior distribution for each of the following sets of observations. In each case, assume a uniform prior for p.**

**(1) W,W,W**

**(2) W,W,W,L**

**(3) L,W,W,L,W,W,W**

In [63]:
def grid_search_posterior_uniform_prior(input_string, n_grid):
  """input_string: experiment output in string form
     n_grid: number of grid points
  """
  input_list = input_string.lower().split(",")
  n = len(input_list)
  w = len([el for el in input_list if el=="w"])

  #define grid
  p_grid = np.linspace(0, 1, n_grid)
  #define prior
  prior = np.repeat(1, n_grid)
  # compute likelihood at each value in grid
  likelihood = stats.binom.pmf(w, n, p_grid)
  # compute product of likelihood and prior
  unstd_posterior = prior * likelihood
  # standardize the posterior, so it sums to 1
  posterior = unstd_posterior / sum(unstd_posterior)

  df = pd.DataFrame()
  df["x"] = p_grid
  df["y"] = posterior

  posterior_chart = alt.Chart(df, title=\
  "Posterior probability with {} measurements and {} grid points and uniform prior"\
            .format(n, n_grid))\
            .mark_line().encode(
                              x="x",
                              y="y"
  )

  return posterior_chart

In [67]:
grid_search_posterior_uniform_prior("W,W,W", 100)

alt.Chart(...)

In [68]:
grid_search_posterior_uniform_prior("W,W,W,L", 100)

alt.Chart(...)

In [69]:
grid_search_posterior_uniform_prior("L,W,W,L,W,W,W", 100)

alt.Chart(...)

**2M2. Now assume a prior for p that is equal to zero when p < 0.5 and is a positive constant when p ≥ 0.5. Again compute and plot the grid approximate posterior distribution for each of the sets of observations in the problem just above.**

In [70]:
def grid_search_posterior_interval_prior(input_string, const, n_grid):
  """input_string: experiment output in string form
     const: positive contant for prior
     n_grid: number of grid points
  """
  if (const < 0) or (const > 1):
      print("Contant should be in [0,1] interval")
  
  input_list = input_string.lower().split(",")
  n = len(input_list)
  w = len([el for el in input_list if el=="w"])

  #define grid
  p_grid = np.linspace(0, 1, n_grid)
  #define prior
  prior = np.where(p_grid < 0.5, 0, const)
  # compute likelihood at each value in grid
  likelihood = stats.binom.pmf(w, n, p_grid)
  # compute product of likelihood and prior
  unstd_posterior = prior * likelihood
  # standardize the posterior, so it sums to 1
  posterior = unstd_posterior / sum(unstd_posterior)

  df = pd.DataFrame()
  df["x"] = p_grid
  df["y"] = posterior

  posterior_chart = alt.Chart(df, title=\
  "Posterior probability with {} measurements and {} grid points and interval prior"\
            .format(n, n_grid))\
            .mark_line().encode(
                              x="x",
                              y="y"
  )

  return posterior_chart

In [71]:
grid_search_posterior_interval_prior("W,W,W", 0.5, 100)

alt.Chart(...)

In [72]:
grid_search_posterior_interval_prior("W,W,W,L", 0.5, 100)

alt.Chart(...)

In [73]:
grid_search_posterior_interval_prior("L,W,W,L,W,W,W", 0.5, 100)

alt.Chart(...)

**2M3. Suppose there are two globes, one for Earth and one for Mars. The Earth globe is 70% covered in water. The Mars globe is 100% land. Further suppose that one of these globes—you don’t know which—was tossed in the air and produced a “land” observation. Assume that each globe was equally likely to be tossed. Show that the posterior probability that the globe was the Earth, conditional on seeing “land” (Pr(Earth|land)), is 0.23.**

In [74]:
display(Math(r"Pr(Earth|land) = \frac{Pr(land|Earth) * Pr(Earth)}{Pr(land)} = \
             \frac{(1-0.7) * 0.5}{0.5 * 0.3 + 0.5 * 1} = \frac {0.15}{0.15 + 0.5} = \
             \frac{0.15}{0.65}\approx 0.23"))

<IPython.core.display.Math object>

**2M4. Suppose you have a deck with only three cards. Each card has two sides, and each side is either black or white. One card has two black sides. The second card has one black and one white side. The third card has two white sides. Now suppose all three cards are placed in a bag and shuffled. Someone reaches into the bag and pulls out a card and places it flat on a table. A black side is shown facing up, but you don’t know the color of the side facing down. Show that the probability that the other side is also black is 2/3. Use the counting method (Section 2 of the chapter) to approach this problem. This means counting up the ways that each card could produce the observed data (a black side facing up on the table).**

We are looking for the probability of extracting black-black card under possible conjectures and 1 draw outcome.

ways to produce black side under each conjecture:

1.   black-black card: 2
2.   black-white card: 1
3.   white-white card: 0

Ways to produce each card: 1,

plausability of producing each card: 1/3.

Posterior probability vector = prior * plausibility = (2, 1, 0) * (1, 1, 1) = (2, 1, 0) => normalized to (2/3, 1/3, 0), where 2/3 defines probability of drawing black card and, respectively, the other side black outcome.




**2M5. Now suppose there are four cards: B/B, B/W, W/W, and another B/B. Again suppose a card is drawn from the bag and a black side appears face up. Again calculate the probability that the other side is black.**

In [75]:
prior = np.array([2, 1, 0, 2])
plausibility = np.array(4*[0.25])
posterior = prior * plausibility
posterior_normed = posterior/sum(posterior)
prob_other_side_black = posterior_normed[0] + posterior_normed[3]
print("Probibility that the other side is black under problem 2M5 conditions is {}"\
      .format(prob_other_side_black))

Probibility that the other side is black under problem 2M5 conditions is 0.8


**2M6. Imagine that black ink is heavy,and so cards with black sides are heavier than cards with white sides. As a result, it’s less likely that a card with black sides is pulled from the bag. So again assume there are three cards: B/B, B/W, and W/W. After experimenting a number of times, you conclude that for every way to pull the B/B card from the bag, there are 2 ways to pull the B/W card and 3 ways to pull the W/W card. Again suppose that a card is pulled and a black side appears face up. Show that the probability the other side is black is now 0.5. Use the counting method, as before.**

In [76]:
#it does not matter if we norm the plausability
prior = np.array([2, 1, 0])
plausibility = np.array([1, 2, 3])
#plausibility_normed = plausibility/sum(plausibility)
posterior = prior * plausibility#_normed
posterior_normed = posterior/sum(posterior)
print("Probibility that the other side is black under problem 2M6 conditions is {}"\
      .format(posterior_normed[1]))

Probibility that the other side is black under problem 2M6 conditions is 0.5


**2M7. Assume again the original card problem, with a single card showing a black side face up. Before looking at the other side, we draw another card from the bag and lay it face up on the table. The face that is shown on the new card is white. Show that the probability that the first card, the one showing a black side, has black on its other side is now 0.75. Use the counting method, if you can. Hint: Treat this like the sequence of globe tosses, counting all the ways to see each observation, for each possible first card.**

In [77]:
#We are looking for the probability of produced sequence of cards is bb-bw or bb-ww. 
#Prior here is number of ways to produce black side on the first step
prior = np.array([2, 1, 0])
#Plausibility is number of ways to get white side on the second step given
#the first step outcome
plausability = np.array([3, 2, 1])
posterior = prior * plausability
posterior_normed = posterior/sum(posterior)
print("Probibility that the other side is black under problem 2M7 conditions is {}"\
      .format(posterior_normed[0]))

Probibility that the other side is black under problem 2M7 conditions is 0.75


**2H1. Suppose there are two species of panda bear. Both are equally common in the wild and live in the same places. They look exactly alike and eat the same food, and there is yet no genetic assay capable of telling them apart. They differ however in their family sizes. Species A gives birth to twins 10% of the time, otherwise birthing a single infant. Species B births twins 20% of the time, otherwise birthing singleton infants. Assume these numbers are known with certainty, from many years of field research. Now suppose you are managing a captive panda breeding program. You have a new female panda of unknown species, and she has just given birth to twins. What is the probability that her next birth will also be twins?**

In [78]:
display(Math(r"Pr(second\ twins|first\ twins) = Pr(2nd\ twins\cap Panda\ is\ A\ species|1st\ twins) * \
               Pr(Panda\ is\ A\ species) + Pr(2nd\ twins\cap Panda\ is\ B\ species|1st\ twins) * \
               Pr(Panda\ is\ B\ species) = (1) + (2)"))
display(Math(r"(1) = \frac{Pr(1st\ twins\cap A\cap 2nd\ twins)}{P(1st\ twins)} * P(A) = \
               \frac{0.1^2 * 0.5}{0.5 * 0.1 + 0.5 * 0.2} = \frac{0.01}{0.3}"))
display(Math(r"(2) = \frac{Pr(1st\ twins\cap B\cap 2nd\ twins)}{P(1st\ twins)} * P(B) = \
               \frac{0.2^2 * 0.5}{0.5 * 0.1 + 0.5 * 0.2} = \frac{0.04}{0.3}"))
display(Math(r"(1) + (2) = \frac{0.05}{0.3} = \approx 0.17"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

**2H2. Recall all the facts from the problem above. Now compute the probability that the panda we have is from species A, assuming we have observed only the first birth and that it was twins.**

In [79]:
#equal chance that panda is either species
prior = np.array([1, 1])
plausability = np.array([0.1, 0.2])
posterior = prior * plausability
posterior_normed = posterior/sum(posterior)
print("Probibility that panda is A species under problem 2H2 conditions is {}"\
      .format(posterior_normed[0]))

Probibility that panda is A species under problem 2H2 conditions is 0.3333333333333333


**2H3. Continuing on from the previous problem,suppose the same panda mother has a second birth and that it is not twins, but a singleton infant. Compute the posterior probability that this panda is species A.**

In [80]:
#prior for this problem is the posterior from the previous one
prior = posterior
#singleton this time
plausability = np.array([0.9, 0.8])
posterior = prior * plausability
posterior_normed = posterior/sum(posterior)
print("Probibility that panda is A species under problem 2H3 conditions is {}"\
      .format(posterior_normed[0]))

Probibility that panda is A species under problem 2H3 conditions is 0.36


**2H4. A common boast of Bayesian statisticians is that Bayesian inference makes it easy to use all of the data, even if the data are of different types.
So suppose now that a veterinarian comes along who has a new genetic test that she claims can identify the species of our mother panda. But the test, like all tests, is imperfect. This is the information you have about the test:**

• The probability it correctly identifies a species A panda is 0.8.

• The probability it correctly identifies a species B panda is 0.65.

**(a) The vet administers the test to your panda and tells you that the test is positive for species A. First ignore your previous information from the births and compute the posterior probability that your panda is species A. (b) Then redo your calculation, now using the birth data as well.**

In [81]:
#(a)
display(Math(r"Pr(Panda\ A|test\ result)= \frac{Pr(test\ result|Panda\ A)}{P(test\ result)} * \
               Pr(Panda A) = \frac{0.8*0.5}{(0.5*0.8 + 0.5*0.35)} = \frac{0.8}{1.15}\approx 0.7"))

<IPython.core.display.Math object>

In [82]:
#alternatively:
prior = np.array([1, 1])
likelihood = np.array([0.8, 1-0.65])
posterior = prior * likelihood
posterior_normed = posterior/sum(posterior)
print("Probibility that panda is A species under problem 2H4 (a) conditions is {}"\
      .format(posterior_normed[0]))

Probibility that panda is A species under problem 2H4 (a) conditions is 0.6956521739130436


In [83]:
#(b)
#again prior is posterior from the previous problem,
#does not matter which event happened first, test or birth
prior = posterior
likelihood = np.array([0.1, 0.2])
posterior = prior * likelihood
posterior_normed = posterior/sum(posterior)
print("Probibility that panda is A species under problem 2H4 (b) conditions is {}"\
      .format(posterior_normed[0]))

Probibility that panda is A species under problem 2H4 (b) conditions is 0.5333333333333333


In [84]:
# with pm.Model() as normal_aproximation:
#     p = pm.Uniform('p', 0, 1)
#     w = pm.Binomial('w', n=total_observations, p=p, observed=water_observations)
#     mean_q = pm.find_MAP()
#     std_q = ((1/pm.find_hessian(mean_q, vars=[p]))**0.5)[0]
# mean_q['p'], std_q